In [1]:
import scala.io.Source

import scala.io.Source

In [2]:
val commits = Source.fromFile("commits").getLines().toArray.toSeq

commits: Seq[String] = Array(
  "shapeless: topic/lazyextensions",
  "3fd06cd0 0. master",
  "0fa6a9d4 1. v0",
  "2085addf 2. Strict Implicit",
  "22b85cb8 3. inlining",
  "cd05b4af 4. cached",
  "0760706f 5. Don't clear scalac implicit cache",
  "41f6f6c5 6. Revert inlining",
  "",
  "upickle-pprint: topic/optim",
  "19052c3f 0. master",
  "2a63bef3 1. Lazy only, Implicit, no Strict or Cached",
  "4266949f 2. No Lazy tails",
  "aabcc2a4 3. Helper type classes for tails",
...

In [3]:
val (Seq(_, shapelessCommits @ _*), Seq(_, _, upickleCommits @ _*)) = commits
  .map(_.split(" ", 2).toSeq)
  .collect{
    case Seq(c, d) => c -> d
    case Seq(other) => other -> ""
  }
  .splitAt(commits.indexWhere(_.isEmpty))

shapelessCommits: Seq[(String, String)] = ArrayBuffer(
  ("3fd06cd0", "0. master"),
  ("0fa6a9d4", "1. v0"),
  ("2085addf", "2. Strict Implicit"),
  ("22b85cb8", "3. inlining"),
  ("cd05b4af", "4. cached"),
  ("0760706f", "5. Don't clear scalac implicit cache"),
  ("41f6f6c5", "6. Revert inlining")
)
upickleCommits: Seq[(String, String)] = ArrayBuffer(
  ("19052c3f", "0. master"),
  ("2a63bef3", "1. Lazy only, Implicit, no Strict or Cached"),
  ("4266949f", "2. No Lazy tails"),
  ("aabcc2a4", "3. Helper type classes for tails"),
  ("aa82a2ba", "4. Usual implicit prioritisation"),
  ("b823fab0", "5. No superfluous Lazy wrapper"),
  ("e5954868", "6. Using Strict"),
  ("5938da82", "6. Cached"),
  ("b4d36c6e", "7. All Strict replaced by Lazy")
)

In [4]:
val shapelessCommitsMap = shapelessCommits.toMap
val upickleCommitsMap = upickleCommits.toMap

shapelessCommitsMap: Map[String, String] = Map(
  "22b85cb8" -> "3. inlining",
  "0fa6a9d4" -> "1. v0",
  "3fd06cd0" -> "0. master",
  "0760706f" -> "5. Don't clear scalac implicit cache",
  "2085addf" -> "2. Strict Implicit",
  "41f6f6c5" -> "6. Revert inlining",
  "cd05b4af" -> "4. cached"
)
upickleCommitsMap: Map[String, String] = Map(
  "e5954868" -> "6. Using Strict",
  "2a63bef3" -> "1. Lazy only, Implicit, no Strict or Cached",
  "aabcc2a4" -> "3. Helper type classes for tails",
  "b4d36c6e" -> "7. All Strict replaced by Lazy",
  "4266949f" -> "2. No Lazy tails",
  "b823fab0" -> "5. No superfluous Lazy wrapper",
  "aa82a2ba" -> "4. Usual implicit prioritisation",
  "19052c3f" -> "0. master",
  "5938da82" -> "6. Cached"
)

In [5]:
case class SingleRun(
  shapelessCommit: String,
  upickleCommit: String,
  run: Int,
  timeSec: Int
) {
  def key = (shapelessCommit, upickleCommit)
}

object SingleRun {
  def fromLine(line: Seq[String]) = {
    assert(line.length == 4)
    SingleRun(line(0), line(1), line(2).toInt, line(3).toInt)
  }
}

defined class SingleRun
defined object SingleRun

In [6]:
load.ivy("com.twitter" %% "algebird-core" % "0.11.0")

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT from sonatype-snapshots, using Fri Jun 05 10:12:58 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT from sonatype-snapshots, using Thu Jun 04 00:37:50 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT from sonatype-snapshots, using Mon Jun 01 02:54:01 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT



In [7]:
import com.twitter.algebird.{ Moments, Monoid }

import com.twitter.algebird.{ Moments, Monoid }

In [8]:
val stats = Source.fromFile("results-1.csv")
  .getLines()
  .toList
  .map(_.split(';').toSeq)
  .drop(1) // drop header
  .map(SingleRun.fromLine)
  .groupBy(_.key)
  .mapValues{ l =>
    val m = Monoid.sum(l.map(run => Moments(run.timeSec)))
    (m.mean, m.stddev)
  }

stats: Map[(String, String), (Double, Double)] = Map(
  ("0760706f", "b4d36c6e") -> (96.8, 2.227105745132009),
  ("22b85cb8", "e5954868") -> (99.0, 2.756809750418044),
  ("2085addf", "b823fab0") -> (105.0, 1.8973665961010278),
  ("0fa6a9d4", "4266949f") -> (156.4, 2.65329983228432),
  ("cd05b4af", "b4d36c6e") -> (94.2, 1.1661903789690597),
  ("0fa6a9d4", "aa82a2ba") -> (117.8, 2.4819347291981715),
  ("0fa6a9d4", "aabcc2a4") -> (139.8, 3.187475490101844),
...

In [9]:
val upickleStats = shapelessCommits
  .map(t => t -> 
     stats
       .collect{ case ((s, u), stats) if s == t._1 =>
         (u, upickleCommitsMap(u)) -> stats
       }
       .toList
       .sortBy(kv => upickleCommits.indexOf(kv._1))
  )

upickleStats: Seq[((String, String), List[((String, String), (Double, Double))])] = ArrayBuffer(
  (
    ("3fd06cd0", "0. master"),
    List(
      (
        ("19052c3f", "0. master"),
        (32.6, 1.019803902718557)
      )
    )
  ),
  (
    ("0fa6a9d4", "1. v0"),
    List(
      (
        ("19052c3f", "0. master"),
...

In [10]:
val shapelessStats = upickleCommits
  .map(t => t -> 
     stats
       .collect{ case ((s, u), stats) if u == t._1 =>
         (s, shapelessCommitsMap(s)) -> stats
       }
       .toList
       .sortBy(kv => shapelessCommits.indexOf(kv._1))
  )

shapelessStats: Seq[((String, String), List[((String, String), (Double, Double))])] = ArrayBuffer(
  (
    ("19052c3f", "0. master"),
    List(
      (
        ("3fd06cd0", "0. master"),
        (32.6, 1.019803902718557)
      ),
      (
        ("0fa6a9d4", "1. v0"),
        (32.8, 0.7483314773547879)
      ),
      (
        ("2085addf", "2. Strict Implicit"),
        (33.0, 1.0954451150103328)
...

In [11]:
load.ivy(
  "com.github.alexarchambault" %% "uplot" % "0.1.0-SNAPSHOT",
  "com.github.alexarchambault" %% "uplot-highcharts" % "0.1.0-SNAPSHOT"
)

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT from sonatype-snapshots, using Fri Jun 05 10:12:58 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT from sonatype-snapshots, using Thu Jun 04 00:37:50 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT from sonatype-snapshots, using Mon Jun 01 02:54:01 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-api_2.11;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#uplot_2.11;0.1.0-SNAPSHOT from sonatype-snapshots, using Wed Feb 11 00:49:03 CET 2015

	Choosing sonatype-snapshots for com.github.alexarchambault#uplot_2.11;0.1.0-

In [12]:
import uplot._

import uplot._

In [13]:
display.html(
  <div>
    <script src="http://code.highcharts.com/stock/highstock.js"></script>
    <script src="http://code.highcharts.com/stock/modules/exporting.js"></script>
    <script src="http://www.highcharts.com/js/themes/grid.js"></script>
  </div>
)

In [14]:
def plot(
  stats: Seq[((String, String), List[((String, String), (Double, Double))])],
  commits: Seq[(String, String)]
) = Plot(
  data = stats.map{case ((_, k), v) =>
    val map = v.toMap
    Data(
      x = (1 until commits.length).map(_.toDouble),
      y = commits.drop(1).map(c => map.get(c).fold(0.0)(_._1)),
      options = Seq(
        Data.Legend(k)
      )
    )
  },
  options = Seq(
    Plot.YLim(0, 180)
  )
)

defined function plot

In [15]:
val id = "shapelessGraph"
display.html(<div id={id}></div>)
display.js(uplot.Highcharts.plotHighchartsJs(plot(shapelessStats, shapelessCommits), id))

id: String = "shapelessGraph"

In [16]:
val id = "upickleGraph"
display.html(<div id={id}></div>)
display.js(uplot.Highcharts.plotHighchartsJs(plot(upickleStats, upickleCommits), id))

id: String = "upickleGraph"